In [ ]:
import numpy as np
import random
import time
import pandas as pd

# Przygotowanie danych wejściowych

Experiment 1: seed=0, ratio=0.6, range_small=(5, 30), range_large=(80, 110), w=0.5, c1=1.0, c2=1.0 => initial=273, final=268, improve=5

In [ ]:
# Pojemność pudełek
BIN_CAPACITY = 180

# Losowa, ale zdeterminowana lista wag: 1000 elementów (60% małych, 40% dużych)
np.random.seed(0)
small_items = np.random.randint(5, 30, size=600)
large_items = np.random.randint(80, 110, size=400)
ITEM_WEIGHTS = np.concatenate([small_items, large_items])
np.random.shuffle(ITEM_WEIGHTS)
ITEM_WEIGHTS = ITEM_WEIGHTS.tolist()

# Liczba przedmiotów
NUM_ITEMS = len(ITEM_WEIGHTS)

print(f"Liczba elementów: {NUM_ITEMS}")
print(f"Pierwsze 10 wag: {ITEM_WEIGHTS[:10]}")


Liczba elementów: 1000
Pierwsze 10 wag: [18, 17, 106, 107, 5, 91, 8, 7, 99, 6]


# Algorytm roju

In [ ]:
# 🎯 Funkcja dekodująca permutację do rozwiązania bin packing (heurystyka First Fit)
# Dla zadanej permutacji elementów pakuje je kolejno do pojemników tak,
# by zmieściły się bez przekroczenia pojemności BIN_CAPACITY.
def first_fit(permutation):
    bins = []
    for index in permutation:
        item = ITEM_WEIGHTS[index]
        placed = False
        for bin in bins:
            if sum(bin) + item <= BIN_CAPACITY:
                bin.append(item)
                placed = True
                break
        if not placed:
            bins.append([item])
    return bins


# 🐝 Klasa cząstki (particle) używana w algorytmie PSO
class Particle:
    def __init__(self, num_items):
        # Losowa permutacja elementów — to aktualna propozycja rozwiązania
        self.position = list(np.random.permutation(num_items))
        self.velocity = []  # lista ruchów (zamian pozycji w permutacji)
        self.best_position = self.position.copy()  # najlepsza znana permutacja dla tej cząstki
        self.best_cost = self.evaluate(self.position)  # liczba użytych pojemników dla tej permutacji

    # Funkcja celu: liczba pojemników potrzebnych dla danej permutacji
    def evaluate(self, position):
        bins = first_fit(position)
        return len(bins)

    # Aktualizacja "prędkości" (czyli zestawu zamian indeksów, które przybliżają nas do najlepszego rozwiązania)
    def update_velocity(self, global_best_position, w=0.5, c1=1.0, c2=1.0):
        swaps = []

        for i in range(NUM_ITEMS):
            # Zamiana w kierunku własnego najlepszego rozwiązania (eksploatacja)
            if random.random() < c1 and self.position[i] != self.best_position[i]:
                j = self.position.index(self.best_position[i])
                swaps.append((i, j))

            # Zamiana w kierunku najlepszego globalnego rozwiązania (eksploracja)
            if random.random() < c2 and self.position[i] != global_best_position[i]:
                j = self.position.index(global_best_position[i])
                swaps.append((i, j))

        random.shuffle(swaps)
        self.velocity = swaps[:20]  # ograniczamy liczbę zmian w jednej iteracji (siła ruchu)

    # Zastosowanie velocity do pozycji cząstki (czyli faktyczne wykonanie zamian)
    def apply_velocity(self):
        for i, j in self.velocity:
            self.position[i], self.position[j] = self.position[j], self.position[i]

    # Sprawdzenie czy nowe położenie jest lepsze — jeśli tak, aktualizujemy pamięć cząstki
    def update_personal_best(self):
        cost = self.evaluate(self.position)
        if cost < self.best_cost:
            self.best_cost = cost
            self.best_position = self.position.copy()


# 🐦 Główna funkcja algorytmu PSO dla problemu bin packing
def bin_packing_pso(num_particles=80, max_iter=200):
    start_time = time.time()

    # Inicjalizacja populacji cząstek (każda z losową permutacją)
    swarm = [Particle(NUM_ITEMS) for _ in range(num_particles)]

    # Wyznaczamy najlepszą początkową cząstkę w całym roju
    global_best = min(swarm, key=lambda p: p.best_cost)
    global_best_position = global_best.best_position.copy()
    global_best_cost = global_best.best_cost

    # Główna pętla optymalizacji
    for iteration in range(max_iter):
        for particle in swarm:
            particle.update_velocity(global_best_position)  # wyznacz ruch
            particle.apply_velocity()                      # wykonaj ruch
            particle.update_personal_best()                # aktualizuj najlepsze lokalne

        # Po aktualizacji wszystkich cząstek — sprawdź, czy któraś poprawiła globalne minimum
        best_candidate = min(swarm, key=lambda p: p.best_cost)
        if best_candidate.best_cost < global_best_cost:
            global_best_cost = best_candidate.best_cost
            global_best_position = best_candidate.best_position.copy()

        print(f"Iteracja {iteration+1}: najlepsza liczba pojemników = {global_best_cost}")

    end_time = time.time()
    final_bins = first_fit(global_best_position)

    # 📋 Zwrot wyniku jako słownik
    result = {
        "number_of_bins": len(final_bins),
        "bin_contents": final_bins,
        "execution_time_seconds": round(end_time - start_time, 2)
    }
    return result


In [ ]:
# ▶️ Uruchomienie algorytmu
result = bin_packing_pso()

# 🖨️ Prezentacja wyniku
print("\n📦 Ostateczny wynik algorytmu PSO (Bin Packing):")
print(f"🔢 Liczba użytych pojemników: {result['number_of_bins']}")
print(f"⏱️ Czas wykonania: {result['execution_time_seconds']} s\n")
for i, bin in enumerate(result['bin_contents']):
    print(f"🗃️ Pojemnik {i+1}: {bin} (suma: {sum(bin)})")

Iteracja 1: najlepsza liczba pojemników = 273
Iteracja 2: najlepsza liczba pojemników = 273
Iteracja 3: najlepsza liczba pojemników = 273
Iteracja 4: najlepsza liczba pojemników = 273
Iteracja 5: najlepsza liczba pojemników = 272
Iteracja 6: najlepsza liczba pojemników = 272
Iteracja 7: najlepsza liczba pojemników = 272
Iteracja 8: najlepsza liczba pojemników = 272
Iteracja 9: najlepsza liczba pojemników = 272
Iteracja 10: najlepsza liczba pojemników = 272
Iteracja 11: najlepsza liczba pojemników = 272
Iteracja 12: najlepsza liczba pojemników = 272
Iteracja 13: najlepsza liczba pojemników = 272
Iteracja 14: najlepsza liczba pojemników = 272
Iteracja 15: najlepsza liczba pojemników = 271
Iteracja 16: najlepsza liczba pojemników = 271
Iteracja 17: najlepsza liczba pojemników = 271
Iteracja 18: najlepsza liczba pojemników = 271
Iteracja 19: najlepsza liczba pojemników = 271
Iteracja 20: najlepsza liczba pojemników = 271
Iteracja 21: najlepsza liczba pojemników = 271
Iteracja 22: najlepsza

# Algorytm genetyczny


In [ ]:
# 🧬 ALGORYTM GENETYCZNY DLA BIN PACKING

def evaluate_solution(permutation):
    return len(first_fit(permutation))

def tournament_selection(population, fitnesses, tournament_size=3):
    selected = random.sample(list(zip(population, fitnesses)), tournament_size)
    selected.sort(key=lambda x: x[1])
    return selected[0][0]

# ✅ Poprawiona wersja order_crossover
def order_crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))

    child = [None] * size
    child[start:end] = parent1[start:end]

    inserted = set(child[start:end])
    current_index = end

    for i in range(size):
        gene = parent2[(end + i) % size]
        if gene not in inserted:
            child[current_index % size] = gene
            current_index += 1
            inserted.add(gene)

    return child

def mutate(permutation, mutation_rate=0.05):
    for i in range(len(permutation)):
        if random.random() < mutation_rate:
            j = random.randint(0, len(permutation) - 1)
            permutation[i], permutation[j] = permutation[j], permutation[i]

def bin_packing_genetic_algorithm(
    population_size=100,
    generations=200,
    crossover_rate=0.9,
    mutation_rate=0.05
):
    start_time = time.time()

    # 🔄 Inicjalizacja populacji
    population = [list(np.random.permutation(NUM_ITEMS)) for _ in range(population_size)]
    fitnesses = [evaluate_solution(ind) for ind in population]

    best_idx = np.argmin(fitnesses)
    best_solution = population[best_idx]
    best_cost = fitnesses[best_idx]

    for generation in range(generations):
        new_population = []

        while len(new_population) < population_size:
            # 🧬 Selekcja
            parent1 = tournament_selection(population, fitnesses)
            parent2 = tournament_selection(population, fitnesses)

            # 🔗 Krzyżowanie
            if random.random() < crossover_rate:
                child = order_crossover(parent1, parent2)
            else:
                child = parent1.copy()

            # 🌀 Mutacja
            mutate(child, mutation_rate)

            new_population.append(child)

        population = new_population
        fitnesses = [evaluate_solution(ind) for ind in population]

        best_idx = np.argmin(fitnesses)
        if fitnesses[best_idx] < best_cost:
            best_solution = population[best_idx]
            best_cost = fitnesses[best_idx]

        print(f"Generacja {generation+1}: najlepsza liczba pojemników = {best_cost}")

    end_time = time.time()
    final_bins = first_fit(best_solution)

    return {
        "number_of_bins": len(final_bins),
        "bin_contents": final_bins,
        "execution_time_seconds": round(end_time - start_time, 2)
    }

# ▶️ Uruchomienie algorytmu genetycznego
result = bin_packing_genetic_algorithm()

# 🖨️ Prezentacja wyniku
print("\n📦 Ostateczny wynik algorytmu genetycznego (Bin Packing):")
print(f"🔢 Liczba użytych pojemników: {result['number_of_bins']}")
print(f"⏱️ Czas wykonania: {result['execution_time_seconds']} s\n")
for i, bin in enumerate(result['bin_contents']):
    print(f"🗃️ Pojemnik {i+1}: {bin} (suma: {sum(bin)})")

Generacja 1: najlepsza liczba pojemników = 271
Generacja 2: najlepsza liczba pojemników = 271
Generacja 3: najlepsza liczba pojemników = 271
Generacja 4: najlepsza liczba pojemników = 270
Generacja 5: najlepsza liczba pojemników = 270
Generacja 6: najlepsza liczba pojemników = 270
Generacja 7: najlepsza liczba pojemników = 270
Generacja 8: najlepsza liczba pojemników = 270
Generacja 9: najlepsza liczba pojemników = 270
Generacja 10: najlepsza liczba pojemników = 270
Generacja 11: najlepsza liczba pojemników = 270
Generacja 12: najlepsza liczba pojemników = 270
Generacja 13: najlepsza liczba pojemników = 270
Generacja 14: najlepsza liczba pojemników = 270
Generacja 15: najlepsza liczba pojemników = 270
Generacja 16: najlepsza liczba pojemników = 270
Generacja 17: najlepsza liczba pojemników = 270
Generacja 18: najlepsza liczba pojemników = 270
Generacja 19: najlepsza liczba pojemników = 270
Generacja 20: najlepsza liczba pojemników = 270
Generacja 21: najlepsza liczba pojemników = 270
G

# ALGORYTM SA

In [ ]:
import numpy as np
import random
import time

BIN_CAPACITY = 180
np.random.seed(0)
small_items = np.random.randint(5, 30, size=600)
large_items = np.random.randint(80, 110, size=400)
ITEM_WEIGHTS = np.concatenate([small_items, large_items])
np.random.shuffle(ITEM_WEIGHTS)
ITEM_WEIGHTS = ITEM_WEIGHTS.tolist()
NUM_ITEMS = len(ITEM_WEIGHTS)

# Funkcja first-fit
def first_fit(permutation):
    bins = []
    for index in permutation:
        item = ITEM_WEIGHTS[index]
        placed = False
        for bin in bins:
            if sum(bin) + item <= BIN_CAPACITY:
                bin.append(item)
                placed = True
                break
        if not placed:
            bins.append([item])
    return bins

# Simulated Annealing dla Bin Packing
def bin_packing_sa(max_iter=10000, initial_temp=100.0, cooling_rate=0.995):
    start_time = time.time()

    # Start: losowa permutacja
    current_solution = list(np.random.permutation(NUM_ITEMS))
    current_cost = len(first_fit(current_solution))

    best_solution = current_solution.copy()
    best_cost = current_cost

    temperature = initial_temp

    for iteration in range(max_iter):
        # Tworzymy nowego kandydata przez zamianę dwóch losowych elementów
        candidate_solution = current_solution.copy()
        i, j = random.sample(range(NUM_ITEMS), 2)
        candidate_solution[i], candidate_solution[j] = candidate_solution[j], candidate_solution[i]

        candidate_cost = len(first_fit(candidate_solution))

        # Czy zaakceptować nowe rozwiązanie?
        cost_diff = candidate_cost - current_cost
        if cost_diff < 0 or random.random() < np.exp(-cost_diff / temperature):
            current_solution = candidate_solution
            current_cost = candidate_cost

            # Aktualizacja najlepszego znalezionego rozwiązania
            if current_cost < best_cost:
                best_solution = current_solution.copy()
                best_cost = current_cost

        # Schładzanie
        temperature *= cooling_rate

        # Opcjonalny print co 1000 iteracji
        if iteration % 1000 == 0:
            print(f"Iteracja {iteration}: Najlepsza liczba pojemników = {best_cost}")

    end_time = time.time()
    final_bins = first_fit(best_solution)

    result = {
        "number_of_bins": len(final_bins),
        "bin_contents": final_bins,
        "execution_time_seconds": round(end_time - start_time, 2)
    }
    return result

# PRZYKŁAD UŻYCIA:
if __name__ == "__main__":
    result = bin_packing_sa()
    print("\n--- WYNIK SA ---")
    print(f"Liczba pojemników: {result['number_of_bins']}")
    print(f"Czas działania: {result['execution_time_seconds']} sekund")

Iteracja 0: Najlepsza liczba pojemników = 278
Iteracja 1000: Najlepsza liczba pojemników = 275
Iteracja 2000: Najlepsza liczba pojemników = 268
Iteracja 3000: Najlepsza liczba pojemników = 268
Iteracja 4000: Najlepsza liczba pojemników = 268
Iteracja 5000: Najlepsza liczba pojemników = 268
Iteracja 6000: Najlepsza liczba pojemników = 267
Iteracja 7000: Najlepsza liczba pojemników = 267
Iteracja 8000: Najlepsza liczba pojemników = 267
Iteracja 9000: Najlepsza liczba pojemników = 267

--- WYNIK SA ---
Liczba pojemników: 267
Czas działania: 205.9 sekund


In [ ]:
# ▶️ Uruchomienie algorytmu
result = bin_packing_sa()

# 🖨️ Prezentacja wyniku
print("\n📦 Ostateczny wynik algorytmu SA (Bin Packing):")
print(f"🔢 Liczba użytych pojemników: {result['number_of_bins']}")
print(f"⏱️ Czas wykonania: {result['execution_time_seconds']} s\n")
for i, bin in enumerate(result['bin_contents']):
    print(f"🗃️ Pojemnik {i+1}: {bin} (suma: {sum(bin)})")

Iteracja 0: Najlepsza liczba pojemników = 277
Iteracja 1000: Najlepsza liczba pojemników = 271
Iteracja 2000: Najlepsza liczba pojemników = 269
Iteracja 3000: Najlepsza liczba pojemników = 268
Iteracja 4000: Najlepsza liczba pojemników = 268
Iteracja 5000: Najlepsza liczba pojemników = 268
Iteracja 6000: Najlepsza liczba pojemników = 268
Iteracja 7000: Najlepsza liczba pojemników = 268
Iteracja 8000: Najlepsza liczba pojemników = 268
Iteracja 9000: Najlepsza liczba pojemników = 268

📦 Ostateczny wynik algorytmu SA (Bin Packing):
🔢 Liczba użytych pojemników: 268
⏱️ Czas wykonania: 191.53 s

🗃️ Pojemnik 1: [12, 99, 8, 9, 24, 22, 5] (suma: 179)
🗃️ Pojemnik 2: [104, 11, 8, 8, 21, 28] (suma: 180)
🗃️ Pojemnik 3: [96, 14, 23, 16, 8, 10, 8, 5] (suma: 180)
🗃️ Pojemnik 4: [102, 24, 18, 8, 23, 5] (suma: 180)
🗃️ Pojemnik 5: [96, 10, 18, 23, 15, 15] (suma: 177)
🗃️ Pojemnik 6: [85, 89, 6] (suma: 180)
🗃️ Pojemnik 7: [105, 22, 18, 28, 7] (suma: 180)
🗃️ Pojemnik 8: [97, 81] (suma: 178)
🗃️ Pojemnik 9: [